In [ ]:
from ginga import __version__
__version__

In [ ]:
# import ginga remote control module
from ginga.util import grc

In [ ]:
import pandas as pd
import re
import numpy as np
import os

In [ ]:
import lsst.daf.persistence as dafPersist
from pfs.lam.opdb import *


In [ ]:
# host and port parameters can be set in the graphical ui of the plugin.
# These are the defaults
#viewer = grc.RemoteClient('localhost', 9000)

viewer = grc.RemoteClient('ws2.pfs', 9000)
#viewer = grc.RemoteClient('actors-ics.pfs', 9000)

In [ ]:
viewer.host

In [ ]:
# This gives you a handle to the reference viewer shell.
# It is similar to what you can do from the command line with:
#    $ grc ginga ...
#
shell = viewer.shell()

# Define data to be loaded or peaklist

In [ ]:
# either load data to the channel or simply plot the peak list
# but we need to define the image to retrieve lamp, exposure time, ....
load_data = False


repo = "sps"
drpPath = "/data/drp"
repoRoot = os.path.join(f"{drpPath}/{repo}")

rerun = "ginga/detrend"
#rerun = "dcb2"

cam = "r3"

basePath = '/drp/analysis/sm3/'

peaklist = "SM3_LAM_may2022_peakList.csv"

visit = 35453

In [ ]:
experimentId = 1109 
visitStart, visitEnd = getVisitRange(experimentId)
visit = int(visitStart)


In [ ]:
if load_data:
    ginga_channel = 'PeakList'
else:
    ginga_channel = f"{cam.upper()}_DETREND"
print(ginga_channel)

In [ ]:
# the RC plugin in the viewer
shell.add_channel(ginga_channel)
# This gives you a handle to a reference viewer channel.
# It is similar to what you can do from the command line with:
#    $ grc channel ...
#
ch = viewer.channel(ginga_channel)

In [ ]:
arm = cam[0]
specId = int(cam[1])
dataId = dict(arm=arm, spectrograph=specId)

dataId.update(visit=int(visit))

In [ ]:
# define defaut parameters
roi_size = 24
com = True  # Center Of Mass
doBck = True
head = 0
tail = 0
criteria = 'EE5'
verbose = True
doPrint = False




In [ ]:

butler = dafPersist.Butler(os.path.join(f"{drpPath}/{repo}/rerun/{rerun}/"))
#rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
butler.getKeys('raw')
lamps = butler.queryMetadata('raw', ['lamps'], dataId) 
calExp = butler.get("calexp",dataId)

print(f'Start visit {visit} of {cam} with {peaklist}')
print(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")


In [ ]:
[exptime] = butler.queryMetadata('raw', ['exptime'], dataId)

In [ ]:
print(f"{exptime:.0f}")

In [ ]:
lamps

In [ ]:
if load_data:
    ch.load_np("image", calExp.image.array, 'fits', {})

In [ ]:
print(lamps)

In [ ]:
# define CANVAS to plot peak list
canvas = viewer.canvas(ginga_channel)


In [ ]:
canvas.clear()

In [ ]:
peaks = pd.read_csv(peaklist)

In [ ]:
selected_peaks = peaks[peaks.arm == arm]
selected_peaks = selected_peaks[(selected_peaks.element.str.contains('|'.join(re.findall('[A-Z][^A-Z]*', "".join(lamps)))))]


In [ ]:
for (X,Y), group in selected_peaks.groupby(["X","Y"]):
    canvas.add('circle',float(X),float(Y), radius=roi_size, color='red')
        